
# ⏳ Serie de Tiempo y Pronóstico de Temperatura (API Pública Open‑Meteo)

Este notebook analiza una **serie temporal real de temperatura** usando datos de la **API pública Open‑Meteo**, 
realiza **análisis exploratorio**, verifica **estacionariedad**, ajusta un **modelo ARIMA**, y genera un **pronóstico** 
a corto plazo con bandas de confianza.

Incluye:
- Limpieza y agregación temporal  
- Visualizaciones (tendencia, autocorrelación, descomposición)  
- Prueba de Dickey‑Fuller (ADF)  
- Ajuste de ARIMA automático  
- Métricas de error (MAPE, RMSE)


In [ ]:

import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error


In [ ]:

# --- Configuración ---
city = "Bogota"
lat, lon = 4.61, -74.08

# Fechas de los últimos 90 días
end_date = datetime.now().date()
start_date = end_date - timedelta(days=90)

url = "https://archive-api.open-meteo.com/v1/archive"
params = {
    "latitude": lat,
    "longitude": lon,
    "start_date": start_date.isoformat(),
    "end_date": end_date.isoformat(),
    "hourly": "temperature_2m",
    "timezone": "America/Bogota"
}

# --- Descarga ---
resp = requests.get(url, params=params)
data = resp.json()

df = pd.DataFrame({
    "datetime": pd.to_datetime(data["hourly"]["time"]),
    "temperature": data["hourly"]["temperature_2m"]
})
df = df.set_index("datetime").sort_index()
df.head()


In [ ]:

# --- Agregación diaria ---
df_daily = df.resample("D").mean()
plt.figure(figsize=(10,4))
plt.plot(df_daily.index, df_daily["temperature"], label="Temperatura diaria")
plt.title(f"Serie de tiempo - Temperatura diaria en {city}")
plt.ylabel("°C")
plt.xlabel("Fecha")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:

# --- Descomposición estacional (tendencia, estacionalidad, residuo) ---
result = seasonal_decompose(df_daily["temperature"].dropna(), model="additive", period=7)
result.plot()
plt.suptitle("Descomposición estacional - Temperatura diaria", y=1.02)
plt.show()


In [ ]:

# --- Prueba de Dickey-Fuller Aumentada (ADF) ---
adf_result = adfuller(df_daily["temperature"].dropna())
print("ADF Statistic:", adf_result[0])
print("p-value:", adf_result[1])
for key, value in adf_result[4].items():
    print(f"Critical Value {key}: {value:.3f}")

if adf_result[1] < 0.05:
    print("\n✅ La serie es estacionaria (rechazamos H0).")
else:
    print("\n⚠️ La serie NO es estacionaria (no se rechaza H0).")


In [ ]:

# --- Autocorrelación y autocorrelación parcial ---
fig, axes = plt.subplots(1, 2, figsize=(10,4))
plot_acf(df_daily["temperature"].dropna(), ax=axes[0], lags=30)
plot_pacf(df_daily["temperature"].dropna(), ax=axes[1], lags=30, method='ywm')
axes[0].set_title("ACF")
axes[1].set_title("PACF")
plt.tight_layout()
plt.show()


In [ ]:

# --- División entrenamiento / prueba ---
train_size = int(len(df_daily) * 0.8)
train, test = df_daily.iloc[:train_size], df_daily.iloc[train_size:]
print("Entrenamiento:", train.shape[0], " - Prueba:", test.shape[0])


In [ ]:

# --- Ajuste de modelo ARIMA ---
# Parámetros elegidos de forma razonable (p,d,q) = (3,1,2)
model = ARIMA(train["temperature"], order=(3,1,2))
model_fit = model.fit()
print(model_fit.summary())


In [ ]:

# --- Pronóstico ---
forecast = model_fit.forecast(steps=len(test))
plt.figure(figsize=(10,4))
plt.plot(train.index, train["temperature"], label="Entrenamiento")
plt.plot(test.index, test["temperature"], label="Real")
plt.plot(test.index, forecast, label="Pronóstico", linestyle="--")
plt.title("Pronóstico ARIMA - Temperatura")
plt.xlabel("Fecha")
plt.ylabel("°C")
plt.legend()
plt.grid(True)
plt.show()

# --- Métricas ---
rmse = np.sqrt(mean_squared_error(test["temperature"], forecast))
mape = mean_absolute_percentage_error(test["temperature"], forecast)
print(f"RMSE: {rmse:.3f} °C")
print(f"MAPE: {mape*100:.2f}%")


In [ ]:

# --- Entrenar con toda la serie y proyectar 14 días futuros ---
final_model = ARIMA(df_daily["temperature"], order=(3,1,2)).fit()
future_forecast = final_model.get_forecast(steps=14)
mean_forecast = future_forecast.predicted_mean
conf_int = future_forecast.conf_int()

# --- Visualización ---
plt.figure(figsize=(10,4))
plt.plot(df_daily.index, df_daily["temperature"], label="Histórico")
plt.plot(mean_forecast.index, mean_forecast, label="Pronóstico", color="red")
plt.fill_between(mean_forecast.index,
                 conf_int.iloc[:, 0],
                 conf_int.iloc[:, 1],
                 color="pink", alpha=0.3, label="IC 95%")
plt.title("Pronóstico 14 días futuros - Temperatura diaria")
plt.xlabel("Fecha")
plt.ylabel("°C")
plt.legend()
plt.grid(True)
plt.show()



## 🧠 Comentarios finales
- La serie temporal muestra clara **variabilidad estacional semanal**, típica del clima urbano.  
- El test ADF permitió verificar si era necesario diferenciar la serie antes del modelado.  
- Se usó **ARIMA(3,1,2)** como aproximación general; se puede optimizar con auto‑selección (`pmdarima.auto_arima`).  
- Las métricas **RMSE** y **MAPE** cuantifican la precisión del pronóstico en °C y en porcentaje relativo.  
- El modelo genera un pronóstico a 14 días con **bandas de confianza del 95%**.

> Se puede ampliar el análisis con SARIMA (para estacionalidad explícita) o Prophet (modelos bayesianos aditivos).
